# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [16]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-edb024393e-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Companies Workflow Series ("Economy of EU States") 

Consider the following exploratory information need:

> Compare businesses across different sectors and types in E.U. countries

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`      | country       | predicate |
| `wd:Q458`      | E.U.         | node |
| `wd:Q142`      | France       | node      |
| `wd:Q4830453`  | Business     | node      |
| `wd:Q6881511`  | Enterprise   | node      |
| `wd:Q29110228` | AXA          | node |
| `wd:Q43183`    | insurance    | node |




Also consider

```
{ 
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q4830453  . 
}



```

is the BGP to retrieve all **french enterprises and businesses**

## Workload Goals

1. Identify the BGP for obtaining the type, legal form, and industry of a company

2. Identify the BGP to retrieve all companies owned by a company located in a EU country

3. Which company has the largest presence in E.U.?

4. Companies have different 'legal forms', compare the number of companies divided in different legal forms

5. Analyze the number of companies per type, legal form, and industry in each state
 
   5.1 What are the top-3 legal form in E.U.? 
   
   5.2 For which companies is defined some form of income or market capitalization or total assets? What is the min, max, and average in each country for a given legal form?
   
   5.3 Which business in each country owns more businesses in other E.U. countries?
   
   5.4 What can we say about industry sectors in various countries?


In [3]:
# start your workflow here

queryString = """
SELECT COUNT(?p)
WHERE { 

{ 
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q4830453  . 
}
} 
"""

print("Results")
run_query(queryString)

# My Workflow


In [4]:
queryString = """
SELECT COUNT(?p)
WHERE { 

{ 
?p wdt:P17 wd:Q142. 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q4830453  . 
}
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '4577')]


1

### 1. Identify the BGP for obtaining the type, legal form, and industry of a company


### Search for properties of business and enterprises

In [5]:
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname WHERE {  
    ?s  wdt:P31 ?type ;
        ?p ?o .
    
    ?p <http://schema.org/name> ?pname . 
    ?o <http://schema.org/name> ?oname . 
            
    FILTER ((?type = wd:Q4830453) || (?type = wd:Q6881511)) .
} 
LIMIT 100
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pname', 'owned by'), ('o', 'http://www.wikidata.org/entity/Q100343342'), ('oname', 'Lubomír Stoklásek')]
[('p', 'http://www.wikidata.org/prop/direct/P169'), ('pname', 'chief executive officer'), ('o', 'http://www.wikidata.org/entity/Q100343342'), ('oname', 'Lubomír Stoklásek')]
[('p', 'http://www.wikidata.org/prop/direct/P355'), ('pname', 'subsidiary'), ('o', 'http://www.wikidata.org/entity/Q54965304'), ('oname', 'Agrostroj International')]
[('p', 'http://www.wikidata.org/prop/direct/P355'), ('pname', 'subsidiary'), ('o', 'http://www.wikidata.org/entity/Q58490622'), ('oname', 'Humpolecké strojírny Humpolec a.s.')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces'), ('o', 'http://www.wikidata.org/entity/Q69392117'), ('oname', 'Orbis')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows'), ('o', 'http://www.wikidata.org/entity/Q69392117'), ('oname', 'Orbis')]
[('p', 'http://www.wikid

100

### 2. Identify the BGP to retrieve all companies owned by a company located in a EU country

As first I identify all companies owned by a something or someone

In [6]:
queryString = """
SELECT ?s ?sname ?o ?owner WHERE {  
    ?s wdt:P31 ?type ;
        wdt:P127 ?o .
    
    ?s <http://schema.org/name> ?sname .
    ?o <http://schema.org/name> ?owner .
    
    FILTER ((?type = wd:Q4830453) || (?type = wd:Q6881511)) .
} 
LIMIT 20
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q223191'), ('sname', 'Adria Airways'), ('o', 'http://www.wikidata.org/entity/Q69349323'), ('owner', '4K KNDNS')]
[('s', 'http://www.wikidata.org/entity/Q4653345'), ('sname', 'APM Terminals B.V'), ('o', 'http://www.wikidata.org/entity/Q18224'), ('owner', 'Maersk')]
[('s', 'http://www.wikidata.org/entity/Q54965304'), ('sname', 'Agrostroj International'), ('o', 'http://www.wikidata.org/entity/Q8183600'), ('owner', 'Agrostroj Pelhřimov')]
[('s', 'http://www.wikidata.org/entity/Q66364316'), ('sname', 'Panorama'), ('o', 'http://www.wikidata.org/entity/Q1755236'), ('owner', 'Civic Forum')]
[('s', 'http://www.wikidata.org/entity/Q8183600'), ('sname', 'Agrostroj Pelhřimov'), ('o', 'http://www.wikidata.org/entity/Q100343342'), ('owner', 'Lubomír Stoklásek')]
[('s', 'http://www.wikidata.org/entity/Q8192634'), ('sname', 'Saga Television Station'), ('o', 'http://www.wikidata.org/entity/Q16746911'), ('owner', 'Nishinippon Shimbun Company')]
[('s', 'http

20

Find EU states

In [9]:
queryString = """
SELECT ?p ?pname ?o ?owner WHERE {  
    wd:Q142 ?p ?o .
    
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?owner .
} 
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1464'), ('pname', 'category for people born here'), ('o', 'http://www.wikidata.org/entity/Q7463334'), ('owner', 'Category:Births in France')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q1050483'), ('owner', 'Loire Valley')]
[('p', 'http://www.wikidata.org/prop/direct/P78'), ('pname', 'top-level Internet domain'), ('o', 'http://www.wikidata.org/entity/Q37271'), ('owner', '.fr')]
[('p', 'http://www.wikidata.org/prop/direct/P163'), ('pname', 'flag'), ('o', 'http://www.wikidata.org/entity/Q43192'), ('owner', 'flag of France')]
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('pname', 'contains administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q161258'), ('owner', 'Clipperton Island')]
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('pname', 'contains administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q25362'), (

272

I see that I need wdt: P463  wd:458

In [28]:
queryString = """
SELECT ?s ?sname ?owner ?country WHERE {  
    ?s wdt:P127 ?o ;
         wdt:P31 ?type .
    ?o wdt:P17 ?c .    
    ?c wdt:P463 wd:Q458 .
    
    ?s <http://schema.org/name> ?sname .
    ?o <http://schema.org/name> ?owner .
    ?c <http://schema.org/name> ?country .
    
    FILTER ((?type = wd:Q4830453) || (?type = wd:Q6881511)) .

} 
LIMIT 20
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q223191'), ('sname', 'Adria Airways'), ('owner', '4K KNDNS'), ('country', 'Germany')]
[('s', 'http://www.wikidata.org/entity/Q4653345'), ('sname', 'APM Terminals B.V'), ('owner', 'Maersk'), ('country', 'Denmark')]
[('s', 'http://www.wikidata.org/entity/Q54965304'), ('sname', 'Agrostroj International'), ('owner', 'Agrostroj Pelhřimov'), ('country', 'Czech Republic')]
[('s', 'http://www.wikidata.org/entity/Q7277940'), ('sname', 'RTÉ News and Current Affairs'), ('owner', 'Raidió Teilifís Éireann'), ('country', 'Ireland')]
[('s', 'http://www.wikidata.org/entity/Q28871958'), ('sname', 'Seier Capital Denmark'), ('owner', 'Seier Capital'), ('country', 'Denmark')]
[('s', 'http://www.wikidata.org/entity/Q15842322'), ('sname', 'Rhenus Office Systems'), ('owner', 'Remondis'), ('country', 'Germany')]
[('s', 'http://www.wikidata.org/entity/Q15842322'), ('sname', 'Rhenus Office Systems'), ('owner', 'Rhenus SE & Co. KG'), ('country', 'Germany')]
[('s', '

20

### 3. Which company has the largest presence in E.U.?




Let's analyze the property of a company, (Q54965304) Agrostroj International

I decide to count the presence based on the assets of a company. 

In [38]:
queryString = """
SELECT ?p ?pname ?o WHERE {   
    wd:Q54965304 ?p ?o .
    
    ?p <http://schema.org/name> ?pname .

} 
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pname', 'owned by'), ('o', 'http://www.wikidata.org/entity/Q8183600')]
[('p', 'http://www.wikidata.org/prop/direct/P1278'), ('pname', 'Legal Entity Identifier'), ('o', '315700WCMMBR35GX2U81')]
[('p', 'http://www.wikidata.org/prop/direct/P1454'), ('pname', 'legal form'), ('o', 'http://www.wikidata.org/entity/Q15646299')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pname', 'headquarters location'), ('o', 'http://www.wikidata.org/entity/Q1000073')]
[('p', 'http://www.wikidata.org/prop/direct/P169'), ('pname', 'chief executive officer'), ('o', 'http://www.wikidata.org/entity/Q100343342')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q213')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q6881511')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http:/

12

I check the company that own this one

In [50]:
queryString = """
SELECT ?p ?pname ?o WHERE {   
    wd:Q8183600 ?p ?o .
    
    ?p <http://schema.org/name> ?pname .
    
} 
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1128'), ('pname', 'employees'), ('o', '2456')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pname', 'owned by'), ('o', 'http://www.wikidata.org/entity/Q100343342')]
[('p', 'http://www.wikidata.org/prop/direct/P1278'), ('pname', 'Legal Entity Identifier'), ('o', '315700454VOYAAE81Q76')]
[('p', 'http://www.wikidata.org/prop/direct/P1454'), ('pname', 'legal form'), ('o', 'http://www.wikidata.org/entity/Q3742494')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pname', 'headquarters location'), ('o', 'http://www.wikidata.org/entity/Q1000073')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received'), ('o', 'http://www.wikidata.org/entity/Q51955134')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q213')]
[('p', 'http://www.wikidata.org/prop/direct/P2139'), ('pname', 'total revenue'), ('o', '6186466000')]
[('p', 'http://www.wikidata.org/

25

I will use P2403

In [51]:
queryString = """
SELECT ?a ?sname WHERE {   
    ?s wdt:P17 ?c ;
        wdt:P2403 ?a .
    ?c wdt:P463 wd:Q458 .
    
    ?s <http://schema.org/name> ?sname .

} 
ORDER BY desc(?a)
LIMIT 1
"""
print("Results")
run_query(queryString)

Results
[('a', '4103786000000'), ('sname', 'MOL Group')]


1

### 4. Companies have different 'legal forms', compare the number of companies divided in different legal forms

In [11]:
queryString = """
SELECT (COUNT (?s) as ?numCompanies) ?legalForm  WHERE {  
    ?s  wdt:P31 ?type ;
        wdt:P1454 ?lF .
    
    ?lF <http://schema.org/name> ?legalForm . 
            
    FILTER ((?type = wd:Q4830453) || (?type = wd:Q6881511)) .
} 
GROUP BY ?legalForm
ORDER BY desc(?numCompanies) 
LIMIT 10
"""
print("Results")
run_query(queryString)


Results
[('numCompanies', '25382'), ('legalForm', 'společnost s ručením omezeným')]
[('numCompanies', '10755'), ('legalForm', 'akciová společnost')]
[('numCompanies', '10245'), ('legalForm', 'joint-stock company')]
[('numCompanies', '4598'), ('legalForm', 'Spoločnosť s ručením obmedzeným')]
[('numCompanies', '3379'), ('legalForm', 'GmbH')]
[('numCompanies', '2619'), ('legalForm', 'public company')]
[('numCompanies', '2305'), ('legalForm', 'private limited liability company')]
[('numCompanies', '2295'), ('legalForm', 'S.A.')]
[('numCompanies', '2102'), ('legalForm', 'Aktiengesellschaft')]
[('numCompanies', '2045'), ('legalForm', 'kabushiki gaisha')]


10

### 5. Analyze the number of companies per type, legal form, and industry in each state  
### 5.1 What are the top-3 legal form in E.U.?

In [54]:
queryString = """
SELECT (COUNT (?s) as ?number) ?legalForm  WHERE {  
    ?s  wdt:P31 ?type ;
        wdt:P1454 ?lF ;
        wdt:P17 ?c .
         
    ?c wdt:P463 wd:Q458 .
    
    ?lF <http://schema.org/name> ?legalForm . 
            
    FILTER ((?type = wd:Q4830453) || (?type = wd:Q6881511)) .
} 
GROUP BY ?legalForm
ORDER BY desc(?number) 
LIMIT 3
"""
print("Results")
run_query(queryString)


Results
[('number', '25395'), ('legalForm', 'společnost s ručením omezeným')]
[('number', '10739'), ('legalForm', 'akciová společnost')]
[('number', '5039'), ('legalForm', 'joint-stock company')]


3

  ### 5.2 For which companies is defined some form of income or market capitalization or total assets? What is the min, max, and average in each country for a given legal form?
   


I will use P1239, P2295 and P2403 that I discover above.

In [67]:
queryString = """
SELECT (AVG(?a) as ?avg) ?sname ?country WHERE {   
    ?s wdt:P17 ?c ;
        wdt:P2403 ?a .
    ?c wdt:P463 wd:Q458 .
    
    ?s <http://schema.org/name> ?sname .
    ?c <http://schema.org/name> ?country .
}
GROUP BY ?sname ?country
ORDER BY desc(?avg)
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('avg', '4103786000000'), ('sname', 'MOL Group'), ('country', 'Hungary')]
[('avg', '2374986000000'), ('sname', 'HSBC'), ('country', 'United Kingdom')]
[('avg', '1475000000000'), ('sname', 'Deutsche Bank'), ('country', 'Germany')]
[('avg', '1458650000000'), ('sname', 'Česká spořitelna'), ('country', 'Czech Republic')]
[('avg', '1378038000000'), ('sname', 'Československá obchodní banka'), ('country', 'Czech Republic')]
[('avg', '1340000000000'), ('sname', 'Banco Santander'), ('country', 'Spain')]
[('avg', '1077334000000'), ('sname', 'Komerční banka'), ('country', 'Czech Republic')]
[('avg', '720000000000'), ('sname', 'Hungarian State Railways'), ('country', 'Hungary')]
[('avg', '707443000000'), ('sname', 'ČEZ Group'), ('country', 'Czech Republic')]
[('avg', '698690000000'), ('sname', 'Bilbao Vizcaya Argentaria Bank'), ('country', 'Spain')]


10

Let's select companies for which is defined some form of income or market capitalization or total assets, with the properties found before.

Firstly, I answer to the query with the companies that has registered the income.

In [86]:
queryString = """
SELECT ?sname ?legalForm ?income ?country WHERE {  
    ?s  wdt:P17 ?c ;
        wdt:P31 ?type ;
        wdt:P1454 ?lF ;
        wdt:P3362 ?income .
        
        ?c wdt:P463 wd:Q458 .
    
    ?s <http://schema.org/name> ?sname . 
    ?lF <http://schema.org/name> ?legalForm .
    ?c <http://schema.org/name> ?country .

    FILTER ((?type = wd:Q4830453) || (?type = wd:Q6881511)) .
} 
GROUP BY ?legalForm
ORDER BY desc(?number) 
LIMIT 20

"""
print("Results")
run_query(queryString)

Results
[('sname', 'PSN'), ('legalForm', 'společnost s ručením omezeným'), ('income', '353195000'), ('country', 'Czech Republic')]
[('sname', 'Fresenius Medical Care - DS'), ('legalForm', 'společnost s ručením omezeným'), ('income', '39179000'), ('country', 'Czech Republic')]
[('sname', 'PRK Partners'), ('legalForm', 'společnost s ručením omezeným'), ('income', '114827000'), ('country', 'Czech Republic')]
[('sname', 'Alca plast'), ('legalForm', 'společnost s ručením omezeným'), ('income', '393516000'), ('country', 'Czech Republic')]
[('sname', 'Atom Centrum'), ('legalForm', 'společnost s ručením omezeným'), ('income', '76399000'), ('country', 'Czech Republic')]
[('sname', 'Cimex Invest'), ('legalForm', 'společnost s ručením omezeným'), ('income', '7739000'), ('country', 'Czech Republic')]
[('sname', 'JULI Motorenwerk'), ('legalForm', 'společnost s ručením omezeným'), ('income', '182844000'), ('country', 'Czech Republic')]
[('sname', 'Arriva City'), ('legalForm', 'společnost s ručením o

20

Then, market capitalization.

In [88]:
queryString = """
SELECT ?sname ?legalForm ?marketCap ?country WHERE {  
    ?s  wdt:P17 ?c ;
        wdt:P31 ?type ;
        wdt:P1454 ?lF ;
        wdt:P2295 ?marketCap .
        
        ?c wdt:P463 wd:Q458 .
    
    ?s <http://schema.org/name> ?sname . 
    ?lF <http://schema.org/name> ?legalForm .
    ?c <http://schema.org/name> ?country .

    FILTER ((?type = wd:Q4830453) || (?type = wd:Q6881511)) .
} 
GROUP BY ?legalForm
ORDER BY desc(?number) 
LIMIT 20

"""
print("Results")
run_query(queryString)

Results
[('sname', 'Spadel'), ('legalForm', 'S.A.'), ('marketCap', '21013000'), ('country', 'Belgium')]
[('sname', 'PKP Intercity'), ('legalForm', 'S.A.'), ('marketCap', '39200000'), ('country', 'Poland')]
[('sname', 'Skils'), ('legalForm', 'společnost s ručením omezeným'), ('marketCap', '63958000'), ('country', 'Czech Republic')]
[('sname', 'Stada Pharma CZ'), ('legalForm', 'společnost s ručením omezeným'), ('marketCap', '10183000'), ('country', 'Czech Republic')]
[('sname', 'Bike Fun International'), ('legalForm', 'společnost s ručením omezeným'), ('marketCap', '54710000'), ('country', 'Czech Republic')]
[('sname', 'IMOS Asset'), ('legalForm', 'společnost s ručením omezeným'), ('marketCap', '177783000'), ('country', 'Czech Republic')]
[('sname', 'SAS Autosystemtechnik'), ('legalForm', 'společnost s ručením omezeným'), ('marketCap', '442883000'), ('country', 'Czech Republic')]
[('sname', 'Heimstaden Czech'), ('legalForm', 'společnost s ručením omezeným'), ('marketCap', '1995733000'), 

20

Lastly, total assets

In [89]:
queryString = """
SELECT ?sname ?legalForm ?totalAssets ?country WHERE {  
    ?s  wdt:P17 ?c ;
        wdt:P31 ?type ;
        wdt:P1454 ?lF ;
        wdt:P2403 ?totalAssets .
        
        ?c wdt:P463 wd:Q458 .
    
    ?s <http://schema.org/name> ?sname . 
    ?lF <http://schema.org/name> ?legalForm .
    ?c <http://schema.org/name> ?country .

    FILTER ((?type = wd:Q4830453) || (?type = wd:Q6881511)) .
} 
GROUP BY ?legalForm
ORDER BY desc(?number) 
LIMIT 20

"""
print("Results")
run_query(queryString)

Results
[('sname', 'Windyty'), ('legalForm', 'Societas Europaea'), ('totalAssets', '104545000'), ('country', 'Czech Republic')]
[('sname', 'CZC.cz'), ('legalForm', 'společnost s ručením omezeným'), ('totalAssets', '883096000'), ('country', 'Czech Republic')]
[('sname', 'Stada Pharma CZ'), ('legalForm', 'společnost s ručením omezeným'), ('totalAssets', '342897000'), ('country', 'Czech Republic')]
[('sname', 'Brembo Czech'), ('legalForm', 'společnost s ručením omezeným'), ('totalAssets', '3849362000'), ('country', 'Czech Republic')]
[('sname', 'Billa'), ('legalForm', 'společnost s ručením omezeným'), ('totalAssets', '7261010000'), ('country', 'Czech Republic')]
[('sname', 'Arriva City'), ('legalForm', 'společnost s ručením omezeným'), ('totalAssets', '585808000'), ('country', 'Czech Republic')]
[('sname', 'Teplárna Kladno'), ('legalForm', 'společnost s ručením omezeným'), ('totalAssets', '11031617000'), ('country', 'Czech Republic')]
[('sname', 'Stock Spirits Group'), ('legalForm', 'publ

20

Couldn't respond properly to the query: I would need to do 2 nested query, but the server responds "endpoint returned code 500 and response." 

### 5.3 Which business in each country owns more businesses in other E.U. countries? 
I use subsidiary property (p355)

In [81]:
queryString = """
SELECT ?sname ?country (COUNT (?sub) as ?numSub) WHERE {  
    ?s  wdt:P17 ?c ;
        wdt:P31 wd:Q4830453 ;
        wdt:P355 ?sub .
        
    ?sub wdt:P17 ?c2 .
    
    ?c wdt:P463 wd:Q458 .
    ?c2 wdt:P463 wd:Q458 .
    
    
    ?s <http://schema.org/name> ?sname .  
    ?c <http://schema.org/name> ?country .
    ?c2 <http://schema.org/name> ?country2 .
    FILTER (?c != ?c2)
    
    
}
GROUP BY ?sname ?country
ORDER BY desc(?numSub) 
LIMIT 30
"""
print("Results")
run_query(queryString)


Results
[('sname', 'ArcelorMittal'), ('country', 'Luxembourg'), ('numSub', '15')]
[('sname', 'Vodafone'), ('country', 'United Kingdom'), ('numSub', '14')]
[('sname', 'Volkswagen Group'), ('country', 'Germany'), ('numSub', '13')]
[('sname', 'Deutsche Telekom'), ('country', 'Germany'), ('numSub', '12')]
[('sname', 'Strabag'), ('country', 'Austria'), ('numSub', '11')]
[('sname', 'Siemens'), ('country', 'Germany'), ('numSub', '10')]
[('sname', 'Phoenix Pharmahandel'), ('country', 'Germany'), ('numSub', '9')]
[('sname', 'RELX Group'), ('country', 'United Kingdom'), ('numSub', '9')]
[('sname', 'Reichswerke Hermann Göring'), ('country', 'Germany'), ('numSub', '9')]
[('sname', 'Orange'), ('country', 'France'), ('numSub', '9')]
[('sname', 'Deutsche Post AG'), ('country', 'Germany'), ('numSub', '8')]
[('sname', 'Thales Group'), ('country', 'France'), ('numSub', '8')]
[('sname', 'International Airlines Group'), ('country', 'United Kingdom'), ('numSub', '7')]
[('sname', 'Kaprain Holdings'), ('coun

30

Couldn't respond properly to the query: I would need to do 2 nested query, but the server responds "endpoint returned code 500 and response." 

  ### 5.4 What can we say about industry sectors in various countries?

Let's show the most spread industry sectors and in which country are they

In [22]:
queryString = """
SELECT (COUNT(?s) as ?numIndustries) ?industry ?country WHERE {
   ?s wdt:P452 ?i ;
       wdt:P17 ?c .
    ?c wdt:P463 wd:Q458 .   
    
    
    ?i <http://schema.org/name> ?industry . 
    ?c <http://schema.org/name> ?country . 
} 
GROUP BY ?industry ?country
ORDER BY desc(?numIndustries)
LIMIT 10

"""
print("Results")
run_query(queryString)

Results
[('numIndustries', '637'), ('industry', 'health care industry'), ('country', 'United Kingdom')]
[('numIndustries', '521'), ('industry', 'brewing'), ('country', 'Germany')]
[('numIndustries', '484'), ('industry', 'Crop production'), ('country', 'United Kingdom')]
[('numIndustries', '465'), ('industry', 'video game industry'), ('country', 'United Kingdom')]
[('numIndustries', '374'), ('industry', 'automotive industry'), ('country', 'Germany')]
[('numIndustries', '373'), ('industry', 'Financial service activities, except insurance and pension funding'), ('country', 'Germany')]
[('numIndustries', '372'), ('industry', 'food and tobacco industry'), ('country', 'Germany')]
[('numIndustries', '344'), ('industry', 'brewing'), ('country', 'Belgium')]
[('numIndustries', '319'), ('industry', 'retail'), ('country', 'United Kingdom')]
[('numIndustries', '312'), ('industry', 'automotive industry'), ('country', 'United Kingdom')]


10